

<h1 style="font-family:'Helvetica'"><b>Data Presentation and Visualization Project<b></h1>


---



1.   Kezia Sekarayu Setyawati - C14170086
2.   Michelle Florensia - C14170087
3.   Adelyn Thungriallu - C14170155


<label>Data Source </label>
<h1> Johns Hopkins University COVID DATA </h1>
<h3> Data Retrieved 11 December 2020 </h3>







In [ ]:
#@title LIBRARY
import json
from datetime import timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
!pip install pmdarima
import pmdarima as pm
from pmdarima.model_selection import train_test_split


# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
#@title LOAD DATA
# Full data
# =========

#MICHI
# full_table = pd.read_csv('gdrive/My Drive/Semester 7/Data Presentation & Visualization/COVID DPnV/data/covid_19_clean_fix.csv')

# day_wise = pd.read_csv('gdrive/My Drive/Semester 7/Data Presentation & Visualization/COVID DPnV/data/day_wise.csv')
# day_wise['Date'] = pd.to_datetime(day_wise['Date'])

# full_grouped = pd.read_csv('gdrive/My Drive/Semester 7/Data Presentation & Visualization/COVID DPnV/data/full_grouped.csv')
# full_grouped['Date'] = pd.to_datetime(full_grouped['Date'])

# country_wise = pd.read_csv('gdrive/My Drive/Semester 7/Data Presentation & Visualization/COVID DPnV/data/country_wise_latest.csv')
# country_wise = country_wise.replace('', np.nan).fillna(0)
# population = pd.read_csv('gdrive/My Drive/Semester 7/Data Presentation & Visualization/COVID DPnV/data/population_by_country_2020.csv')

#KEZIA
full_table = pd.read_csv('gdrive/My Drive/Colab Notebooks/COVID DPnV/data/covid_19_clean_fix.csv')
full_grouped = pd.read_csv('gdrive/My Drive/Colab Notebooks/COVID DPnV/data/full_grouped.csv')
full_grouped['Date'] = pd.to_datetime(full_grouped['Date'])
day_wise = pd.read_csv('gdrive/My Drive/Colab Notebooks/COVID DPnV/data/day_wise.csv')
day_wise['Date'] = pd.to_datetime(day_wise['Date'])
country_wise = pd.read_csv('gdrive/My Drive/Colab Notebooks/COVID DPnV/data/country_wise_latest.csv')
country_wise = country_wise.replace('', np.nan).fillna(0)
population = pd.read_csv('gdrive/My Drive/Colab Notebooks/COVID DPnV/data/population_by_country_2020.csv')
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [3]:
#@title FUNCTION PLOT BY DAY
def plot_daywise(col, hue):
    fig = px.bar(day_wise, x="Date", y=col, width=700, color_discrete_sequence=[hue])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="")
    fig.show()
def plot_daywise_line(col, hue):
    fig = px.line(day_wise, x="Date", y=col, width=700, color_discrete_sequence=[hue])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="")
    fig.show()

In [4]:
#@title CASES OVER TIME
temp = full_grouped.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.area(temp, x="Date", y="Count", color='Case', height=600, width=700,
             title='Cases over time', color_discrete_sequence = [rec, dth, act])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [5]:
#@title FUNCTION PLOT MAP
def plot_map(df, col, pal):
    df = df[df[col]>0]
    df[col] = np.log10(df[col])
    fig = px.choropleth(df, locations="Country/Region", locationmode='country names', 
                  color=col, hover_name="Country/Region", 
                  title=col, hover_data=[col], color_continuous_scale=pal)
#     fig.update_layout(coloraxis_showscale=False)
    fig.show()

In [6]:
#@title MAP CONFIRMED CASES
plot_map(country_wise, 'Confirmed', 'matter')

In [7]:
#@title MAP DEATH PERCENTAGE CASES
plot_map(country_wise, 'Deaths / 100 Cases', 'matter')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
#@title GET POPULATION
population.rename(columns = {'Country (or dependency)':'Country/Region', 'Population (2020)':'Population'}, inplace = True) 
population = population.iloc[:,0:2]
population = population.replace('United States', 'US')
population = population.replace('Czech Republic (Czechia)', 'Czechia')
population = population.replace('Saint Kitts & Nevis', 'Saint Kitts and Nevis')
join = pd.merge(left=country_wise, right=population, how='left', on=['Country/Region'])
join['Confirmed Percentage'] = round((join['Confirmed']/join['Population'])*100, 2)
join = join.dropna()
join

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region,Population,Confirmed Percentage
0,Afghanistan,48053,1935,38099,8019,202,16,67,4.03,79.29,5.08,46837,1216,2.60,Eastern Mediterranean,39074280.0,0.12
1,Albania,46061,965,23609,21487,873,14,537,2.10,51.26,4.09,40501,5560,13.73,Europe,2877239.0,1.60
2,Algeria,90579,2564,59135,28880,565,10,989,2.83,65.29,4.34,85927,4652,5.41,Africa,43984569.0,0.21
3,Andorra,7190,78,6505,607,28,0,53,1.08,90.47,1.20,6904,286,4.14,Europe,77287.0,9.30
4,Angola,15925,362,8679,6884,121,4,100,2.27,54.50,4.17,15361,564,3.67,Africa,33032075.0,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Venezuela,106280,938,101384,3958,428,5,432,0.88,95.39,0.93,103548,2732,2.64,Americas,28421581.0,0.37
187,Vietnam,1385,35,1225,125,4,0,0,2.53,88.45,2.86,1361,24,1.76,Western Pacific,97490013.0,0.00
189,Yemen,2081,606,1383,92,2,0,0,29.12,66.46,43.82,2239,-158,-7.06,Eastern Mediterranean,29935468.0,0.01
190,Zambia,18091,364,17307,420,29,0,12,2.01,95.67,2.10,17730,361,2.04,Africa,18468257.0,0.10


In [9]:
#@title TOP 10 COUNTRY
def plot_hbar(df, col, n, hover_data=[]):
    fig = px.bar(df.sort_values(col).tail(n), 
                 x=col, y="Country/Region", color='WHO Region',  
                 text=col, orientation='h', width=700, hover_data=hover_data,
                 color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.update_layout(title=col, xaxis_title="", yaxis_title="", 
                      yaxis_categoryorder = 'total ascending')
    fig.show()

plot_hbar(country_wise, 'Confirmed', 15)
plot_hbar(country_wise, 'Active', 15)
plot_hbar(country_wise, 'Deaths', 15)
plot_hbar(country_wise, 'Recovered', 15)

In [10]:
#@title COUNTRY CASES GROWTH
def plot_bubble(col, pal):
    temp = full_table[full_table[col]>0].sort_values('Country/Region', ascending=False)
    fig = px.scatter(temp, x='Date', y='Country/Region', size=col, color=col, height=3000,
                    color_continuous_scale=pal)
    fig.update_layout(yaxis = dict(dtick = 1))
    fig.update(layout_coloraxis_showscale=False)
    fig.show()
plot_bubble('Confirmed', 'Viridis')

Output hidden; open in https://colab.research.google.com to view.

In [11]:
#@title PERCENTAGE OF CONFIRMEND CASES BY COUNTRY
fig = px.scatter(join, x='Population', y='Confirmed', color='WHO Region', 
                 height=700, hover_name='Country/Region', log_x=True, log_y=True, 
                 title='Population : Confirmed', size='Confirmed Percentage',
                 color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=True)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [12]:
#@title ARIMA FORECASTING FUNCTION
arima_df = (day_wise
            .loc[:, ["Date", "Confirmed"]] 
            .rename(columns={"Confirmed": "TargetValue"})
            .set_index("Date"))

def fit_predict_auto_arima(df_train, df_test):
    arima = pm.auto_arima(df_train, start_p=0, start_q=0)
    forecasts, forecasts_pred_interval = arima.predict(df_test.shape[0]+300, 
                                                       return_conf_int=True, 
                                                       alpha=0.05)

    forecasts = pd.Series(data=forecasts)
    forecasts_pred_interval = pd.DataFrame(data=forecasts_pred_interval, 
                                           columns=["low", "high"])

    return arima, forecasts, forecasts_pred_interval, arima.order

df_train, df_test = train_test_split(arima_df)

arima_model, forecasts, forecasts_pred_interval, forecast_arima_order = fit_predict_auto_arima(df_train, df_test)

In [13]:
#@title PREDICTION CONFIRMED CASES TILL FEBRUARY
dates = pd.date_range(start='9/21/2020', end='2/1/2021')
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_train.index, y=df_train.TargetValue, 
                         mode='lines+markers', name="train data"))
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.TargetValue, 
                         mode='lines+markers', name="test data"))
fig.add_trace(go.Scatter(x=dates, y=forecasts, 
                         mode='lines+markers', name="forecast"))
fig.add_trace(go.Scatter(x=dates, y=forecasts_pred_interval.low, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="lower prediction interval"))
fig.add_trace(go.Scatter(x=dates, y=forecasts_pred_interval.high, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="upper prediction interval"))

fig.update_layout(hovermode="x",
                  title=f"ARIMA{forecast_arima_order} applied to the number of confirmed cases",
                  xaxis_title="Date",
                  yaxis_title="Number of confirmed daily cases")

fig.show()

In [14]:
#@title RATE OF RECOVERY AND DEATHS
temp = day_wise.groupby('Date')['Recovered / 100 Cases', 'Deaths / 100 Cases' ].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered / 100 Cases', 'Deaths / 100 Cases'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.area(temp, x="Date", y="Count", color='Case', height=600, width=700,
             title='Rate of Recovery and Deaths over time', color_discrete_sequence = [rec, dth, act])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [15]:
#@title PREDICTION FOR MORTALITY RATE
day_wise_df = (day_wise
            .loc[:, ["Date", "Deaths / 100 Cases"]] 
            .rename(columns={"Deaths / 100 Cases": "TargetValue"})
            .set_index("Date"))
df_train, df_test = train_test_split(day_wise_df)
arima_model, forecasts, forecasts_pred_interval, forecast_arima_order = fit_predict_auto_arima(df_train, df_test)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_train.index, y=df_train.TargetValue, 
                         mode='lines+markers', name="train data"))
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.TargetValue, 
                         mode='lines+markers', name="test data"))
fig.add_trace(go.Scatter(x=dates, y=forecasts, 
                         mode='lines+markers', name="forecast"))
fig.add_trace(go.Scatter(x=dates, y=forecasts_pred_interval.low, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="lower prediction interval"))
fig.add_trace(go.Scatter(x=dates, y=forecasts_pred_interval.high, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="upper prediction interval"))

fig.update_layout(hovermode="x",
                  title=f"ARIMA{forecast_arima_order} applied to the Mortality Rate",
                  xaxis_title="Date",
                  yaxis_title="Number of Death / 100 Cases")

fig.show()